In [3]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [4]:
import { z } from "zod";

// 简单使用
const stringSchema = z.string();
stringSchema.parse("Hello, Zod!");

"Hello, Zod!"

In [5]:
stringSchema.parse(2323);

ZodError: [
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "number",
    "path": [],
    "message": "Expected string, received number"
  }
]

In [6]:
// 基础类型
const stringSchema = z.string();
const numberSchema = z.number();
const booleanSchema = z.boolean();

// 数组
const stringArraySchema = z.array(z.string());
stringArraySchema.parse(["apple", "banana", "cherry"]); 

// 对象
const personSchema = z.object({
  name: z.string(),
  age: z.number(),
  // 可选类型
  isStudent: z.boolean().optional(),
  // 默认值
  home: z.string().default("no home")
});

// 联合类型
const mixedTypeSchema = z.union([z.string(), z.number()]);
mixedTypeSchema.parse("hello"); 
mixedTypeSchema.parse(42); 

42

In [7]:
import { z } from "zod";

const getCurrentWeatherSchema = z.object({
  location: z.string().describe("The city and state, e.g. San Francisco, CA"),
  unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
});

In [8]:
import { zodToJsonSchema } from "zod-to-json-schema";

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema)
paramSchema

{
  type: "object",
  properties: {
    location: {
      type: "string",
      description: "The city and state, e.g. San Francisco, CA"
    },
    unit: {
      type: "string",
      enum: [ "celsius", "fahrenheit" ],
      description: "The unit of temperature"
    }
  },
  required: [ "location", "unit" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [9]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema),
            }
        }
    ]
})

await modelWithTools.invoke("北京的天气怎么样");

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_sOosG30LLVyhlJiAU4m65dHa",
          type: "function",
          function: [Object]
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_sOosG30LLVyhlJiAU4m65dHa",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location":"Beijing","unit":"celsius"}'
        }
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 20, promptTokens: 88, totalTokens: 108 },
    finish_reason: "tool_calls"
  }
}

In [10]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelWithTools)

await chain.invoke({
    input: "北京的天气怎么样"
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_iNKb5pxCXlHEJOpnpltJa7Za",
          type: "function",
          function: [Object]
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_iNKb5pxCXlHEJOpnpltJa7Za",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location":"Beijing","unit":"celsius"}'
        }
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 20, promptTokens: 94, totalTokens: 114 },
    finish_reason: "tool_calls"
  }
}

In [11]:
const getCurrentTimeSchema = z.object({
  format: z
    .enum(["iso", "locale", "string"])
    .optional()
    .describe("The format of the time, e.g. iso, locale, string"),
});

zodToJsonSchema(getCurrentTimeSchema)

{
  type: "object",
  properties: {
    format: {
      type: "string",
      enum: [ "iso", "locale", "string" ],
      description: "The format of the time, e.g. iso, locale, string"
    }
  },
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [12]:
const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithMultiTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema)
            }
        },
        {
            type: "function",
            function: {
                name: "getCurrentTime",
                description: "Get the current time in a given format",
                parameters: zodToJsonSchema(getCurrentTimeSchema)
            }
        }
    ]
})

await modelWithMultiTools.invoke("现在几点了？");


AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_7ifAoekTtCmixNI9TIZkCnpl",
          type: "function",
          function: [Object]
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_7ifAoekTtCmixNI9TIZkCnpl",
        type: "function",
        function: { name: "getCurrentTime", arguments: '{"format":"locale"}' }
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 14, promptTokens: 136, totalTokens: 150 },
    finish_reason: "tool_calls"
  }
}

In [13]:
await modelWithMultiTools.invoke("现在 iso 格式的时间是什么？");

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_ADHuT19XeAt1kwUw6CU56br9",
          type: "function",
          function: [Object]
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_ADHuT19XeAt1kwUw6CU56br9",
        type: "function",
        function: { name: "getCurrentTime", arguments: '{"format":"iso"}' }
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 14, promptTokens: 142, totalTokens: 156 },
    finish_reason: "tool_calls"
  }
}

In [14]:
const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithForce = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema)
            }
        },
        {
            type: "function",
            function: {
                name: "getCurrentTime",
                description: "Get the current time in a given format",
                parameters: zodToJsonSchema(getCurrentTimeSchema)
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "getCurrentWeather"
        }
    }
})

await modelWithForce.invoke("现在几点了？");

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_nLTV44E0mSsfsFPvu3PUSRe7",
          type: "function",
          function: [Object]
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_nLTV44E0mSsfsFPvu3PUSRe7",
        type: "function",
        function: { name: "getCurrentWeather", arguments: '{"format":"locale"}' }
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 5, promptTokens: 145, totalTokens: 150 },
    finish_reason: "stop"
  }
}

## Tagging

In [ ]:
const taggingSchema = z.object({
  emotion:z.enum(["pos", "neg", "neutral"]).describe("文本的情感"),
  language: z.string().describe("文本的核心语言（应为ISO 639-1代码）"),
});

In [ ]:
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";

const model = new ChatOpenAI({
    temperature: 0 
})

const modelTagging = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "tagging",
                description: "为特定的文本片段打上标签",
                parameters: zodToJsonSchema(taggingSchema)
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "tagging"
        }
    }
})

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考，你有充足的时间进行严谨的思考，然后按照指示对文本进行标记"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelTagging).pipe(new JsonOutputToolsParser())

In [ ]:
await chain.invoke({
    input: "hello world"
})

In [ ]:
await chain.invoke({
    input: "写代码太难了，👴 不干了"
})

In [ ]:
await chain.invoke({
    // 日语，圣诞快乐
    input: "メリークリスマス!"
})

In [ ]:
await chain.invoke({
    input: "我非常喜欢 AI，特别是 LLM，因为它非常 powerful"
})

## Extraction

In [16]:
const personExtractionSchema = z.object({
    name: z.string().describe("人的名字"),
    age: z.number().optional().describe("人的年龄")
}).describe("提取关于一个人的信息");

const relationExtractSchema = z.object({
    people: z.array(personExtractionSchema).describe("提取所有人"),
    relation: z.string().describe("人之间的关系, 尽量简洁")
})

In [17]:
const schema = zodToJsonSchema(relationExtractSchema)

In [18]:
console.log(schema)

{
  type: "object",
  properties: {
    people: {
      type: "array",
      items: {
        type: "object",
        properties: { name: [Object], age: [Object] },
        required: [ "name" ],
        additionalProperties: false,
        description: "提取关于一个人的信息"
      },
      description: "提取所有人"
    },
    relation: { type: "string", description: "人之间的关系, 尽量简洁" }
  },
  required: [ "people", "relation" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [19]:
console.log(schema.properties.people)

{
  type: "array",
  items: {
    type: "object",
    properties: {
      name: { type: "string", description: "人的名字" },
      age: { type: "number", description: "人的年龄" }
    },
    required: [ "name" ],
    additionalProperties: false,
    description: "提取关于一个人的信息"
  },
  description: "提取所有人"
}


In [20]:
const model = new ChatOpenAI({
    temperature: 0 
})

const modelExtract = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "relationExtract",
                description: "提取数据中人的信息和人的关系",
                parameters: zodToJsonSchema(relationExtractSchema)
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "relationExtract"
        }
    }
})

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考，你有充足的时间进行严谨的思考，然后提取文中的相关信息，如果没有明确提供，请不要猜测，可以仅提取部分信息"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelExtract).pipe(new JsonOutputToolsParser())

ReferenceError: JsonOutputToolsParser is not defined

In [ ]:
await chain.invoke({
    input: "小明现在 18 岁了，她妈妈是小丽"
})

In [ ]:
await chain.invoke({
    input: "我是小明现在 18 岁了，我和小 A、小 B 是好朋友，都一样大"
})

In [ ]:
await chain.invoke({
    input: "我是小明"
})